In [1]:
import re,requests,math,time
from random import randint
from bs4 import BeautifulSoup
import pandas as pd
import json
from collections import defaultdict

In [2]:
# set type of address

url0 ='https://www.mobile01.com/' #mobile01 homepage
url1 ='waypointtopiclist.php?f=405&p='  
url2 = url0 + 'waypointtopicdetail.php?f=405&'  #mobile01 travel homepage
first_page_url = url0 + url1

In [3]:
first_page_url  # preview first_page_url

'https://www.mobile01.com/waypointtopiclist.php?f=405&p='

In [ ]:
# definite a function for getting all pages link

def get_all_pages_link(first_page_url):
    links = []
    for i in range(1,int(max_page)+1):
        links.append(first_page_url + str(i))
    return links

In [ ]:
# get the max_page of this section
res = requests.get(first_page_url)
soup = BeautifulSoup(res.text,'lxml')
max_page = soup.select('p.numbers')[0].text.split('共')[1].split('頁')[0]  

# definite a function for getting all topic links
def get_all_topic_links(first_page_url):
    links = []
    page = 1
    for url in get_all_pages_link(first_page_url):
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'lxml')
        
        # because article link contain top and gen(stick-top topic and normal topic) 
        # use try and except to avoid error(link is not exist)
        try:
            top_links = []
            for i in [i for i in soup.select('span > a.topic_top')]:
                top_links.append('https://www.mobile01.com/waypointtop' + i['href'].split('top')[1])
            gen_links = []
            for i in [i for i in soup.select('span > a.topic_gen')]:
                gen_links.append('https://www.mobile01.com/waypointtop' + i['href'].split('top')[1])
            print('page{} is OK!'.format(page))
            links += top_links + gen_links
        except:
            gen_links = []
            for i in [i for i in soup.select('span > a.topic_gen')]:
                gen_links.append('https://www.mobile01.com/waypointtop' + i['href'].split('top')[1])
            print('page{} is OK!'.format(page))
            links +=gen_links
        time.sleep(1)
        page+=1
    return links
        

In [ ]:
all_topic_links = get_all_topic_links(first_page_url)  #preview all topic links

In [ ]:
### save all topic links to avoid to continue post request to the server  ###

# for i in all_page_links:
#     f = open("C:\\Users\\Java\\Desktop\\links.text", "a",encoding='utf8')   #workspace
#     f.write(i+'\n')
#     f.close()

In [5]:
### read all topic links from workspace ###

# f = open("C:\\Users\\Java\\Desktop\\links.text", "r",encoding='utf8')  #workspace
# all_link = []
# for i in f:
#     all_link.append(i)

In [8]:
### preview all link ###

# len(all_link)

7702

In [ ]:
# inside_page_link = url0 + soup.select('span > a.topic_gen')[0]['href'] + '&p='

In [ ]:
# url = inside_page_link
# res = requests.get(url)
# soup = BeautifulSoup(res.text,'lxml')
# max_inside_page = int(soup.select('.contentfoot')[0].text.split('共')[1].split('頁')[0])

In [57]:
### start this function to get all topic and save to the workspace ##

start_topic =1
test = {}
for link in all_topic_links:
    url = link + '&p='
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'lxml')
    max_inside_page = int(soup.select('.contentfoot')[0].text.split('共')[1].split('頁')[0])
    
    author_info = soup.select('div > .single-post-author')[0].text.strip().replace('\n\n','\n').replace('私訊連結','').split('\n')
#     try:
    for s in author_info:
        if s == '無圖示': 
            author_info.remove(s)
    test['作者'] = author_info[0]
    test['標題'] = soup.select('.topic')[0].text 
    test['發文時間'] = author_info[2].split(' ')[0]
    test['人氣'] = author_info[1].split(' ')[1].replace(',','')
    test['來源'] = "mobile01"
    
    test['文章內容'] = soup.select('div.single-post')[0].text.split('\n\n\n\n')[1]
    test['url'] = link
     
    #     

    # #     

    total_author =soup.select('div > .fn')
    
    cm1 =[]
    for i in range(1,len(total_author)):
        author_info = soup.select('div > .single-post-author')[i].text.strip().replace('\n\n','\n').replace('私訊連結','').split('\n')

        
        for s in author_info:
            if s == '無圖示': 
                author_info.remove(s)

        author = author_info[0]
        article_date = author_info[2].split(' ')[0]
        content = soup.select('div.single-post')[i].text.split('\n\n\n\n')[1]
        dd ={'作者':author,'發文日期':article_date,'回文內容':content}
        cm1.append(dd)
    
    time.sleep(randint(3,5))
    

    t2_author = []
    t2_date = []
    t2_content = []
    cm2 = []
    for i in range(2,max_inside_page+1):
        inside_page_url = url + str(i)
        res = requests.get(inside_page_url)
        soup = BeautifulSoup(res.text,'lxml')
        total_author =soup.select('div > .fn')
        
          
        
        for j in range(0,len(total_author)):
            author_info = soup.select('div > .single-post-author')[j].text.strip().replace('\n\n','\n').replace('私訊連結','').split('\n')
            for s in author_info:
                if s == '無圖示': 
                    author_info.remove(s)
            
            author = author_info[0]
            article_date = author_info[2].split(' ')[0]
            content = soup.select('div.single-post')[j].text.split('\n\n\n\n')[1].split('\n\n\n')[0]
#             try:
#                 if re.search('(恕刪)',content):
#                     content = soup.select('div.single-post')[j].text.split('\n\n\n\n')[1].split('(恕刪)')[1]
#                 else:
#                     content = soup.select('div.single-post')[j].text.split('\n\n\n\n')[1]
#             except:
#                 pass
            dd ={'作者':author,'發文日期':article_date,'回文內容':content}
            
            cm2.append(dd)
  
        time.sleep(randint(3,5))
    cm = cm1 + cm2
    
    test['回文'] = cm
    
      

 

    
    dest_path = "C:\\Users\\Java\\Desktop\\mobile01_JP\\command{}.text".format(start_topic)
    with open(dest_path,'a',encoding="utf-8") as f:
#         f.write(str(test))
        f.write(json.dumps(test,sort_keys=True,ensure_ascii=False))
        f.close()
    
    print('第{}則文章以爬取完成'.format(start_topic))
    time.sleep(randint(1,3)) 
    start_topic += 1       

第1208則文章以爬取完成
第1209則文章以爬取完成
第1210則文章以爬取完成
第1211則文章以爬取完成
第1212則文章以爬取完成
第1213則文章以爬取完成
第1214則文章以爬取完成
第1215則文章以爬取完成
第1216則文章以爬取完成
第1217則文章以爬取完成
第1218則文章以爬取完成
第1219則文章以爬取完成
第1220則文章以爬取完成
第1221則文章以爬取完成
第1222則文章以爬取完成
第1223則文章以爬取完成
第1224則文章以爬取完成
第1225則文章以爬取完成
第1226則文章以爬取完成
第1227則文章以爬取完成
第1228則文章以爬取完成
第1229則文章以爬取完成
第1230則文章以爬取完成
第1231則文章以爬取完成
第1232則文章以爬取完成
第1233則文章以爬取完成
第1234則文章以爬取完成
第1235則文章以爬取完成
第1236則文章以爬取完成
第1237則文章以爬取完成
第1238則文章以爬取完成
第1239則文章以爬取完成
第1240則文章以爬取完成
第1241則文章以爬取完成
第1242則文章以爬取完成
第1243則文章以爬取完成
第1244則文章以爬取完成
第1245則文章以爬取完成
第1246則文章以爬取完成
第1247則文章以爬取完成
第1248則文章以爬取完成
第1249則文章以爬取完成
第1250則文章以爬取完成
第1251則文章以爬取完成
第1252則文章以爬取完成
第1253則文章以爬取完成
第1254則文章以爬取完成
第1255則文章以爬取完成
第1256則文章以爬取完成
第1257則文章以爬取完成
第1258則文章以爬取完成
第1259則文章以爬取完成
第1260則文章以爬取完成
第1261則文章以爬取完成
第1262則文章以爬取完成
第1263則文章以爬取完成
第1264則文章以爬取完成
第1265則文章以爬取完成
第1266則文章以爬取完成
第1267則文章以爬取完成
第1268則文章以爬取完成
第1269則文章以爬取完成
第1270則文章以爬取完成
第1271則文章以爬取完成
第1272則文章以爬取完成
第1273則文章以爬取完成
第1274則文章以爬取完成
第1275則文章以爬取完成
第1276則文章以爬取完成
第1277則文章以爬取完成
第1278則文章以爬取完成
第1279則

第1794則文章以爬取完成
第1795則文章以爬取完成
第1796則文章以爬取完成
第1797則文章以爬取完成
第1798則文章以爬取完成
第1799則文章以爬取完成
第1800則文章以爬取完成
第1801則文章以爬取完成
第1802則文章以爬取完成
第1803則文章以爬取完成
第1804則文章以爬取完成
第1805則文章以爬取完成
第1806則文章以爬取完成
第1807則文章以爬取完成
第1808則文章以爬取完成
第1809則文章以爬取完成
第1810則文章以爬取完成
第1811則文章以爬取完成
第1812則文章以爬取完成
第1813則文章以爬取完成
第1814則文章以爬取完成
第1815則文章以爬取完成
第1816則文章以爬取完成
第1817則文章以爬取完成
第1818則文章以爬取完成
第1819則文章以爬取完成
第1820則文章以爬取完成
第1821則文章以爬取完成
第1822則文章以爬取完成
第1823則文章以爬取完成
第1824則文章以爬取完成
第1825則文章以爬取完成
第1826則文章以爬取完成
第1827則文章以爬取完成
第1828則文章以爬取完成
第1829則文章以爬取完成
第1830則文章以爬取完成
第1831則文章以爬取完成
第1832則文章以爬取完成
第1833則文章以爬取完成
第1834則文章以爬取完成
第1835則文章以爬取完成
第1836則文章以爬取完成
第1837則文章以爬取完成
第1838則文章以爬取完成
第1839則文章以爬取完成
第1840則文章以爬取完成
第1841則文章以爬取完成
第1842則文章以爬取完成
第1843則文章以爬取完成
第1844則文章以爬取完成
第1845則文章以爬取完成
第1846則文章以爬取完成
第1847則文章以爬取完成
第1848則文章以爬取完成
第1849則文章以爬取完成
第1850則文章以爬取完成
第1851則文章以爬取完成
第1852則文章以爬取完成
第1853則文章以爬取完成
第1854則文章以爬取完成
第1855則文章以爬取完成
第1856則文章以爬取完成
第1857則文章以爬取完成
第1858則文章以爬取完成
第1859則文章以爬取完成
第1860則文章以爬取完成
第1861則文章以爬取完成
第1862則文章以爬取完成
第1863則文章以爬取完成
第1864則文章以爬取完成
第1865則

第2380則文章以爬取完成
第2381則文章以爬取完成
第2382則文章以爬取完成
第2383則文章以爬取完成
第2384則文章以爬取完成
第2385則文章以爬取完成
第2386則文章以爬取完成
第2387則文章以爬取完成
第2388則文章以爬取完成
第2389則文章以爬取完成
第2390則文章以爬取完成
第2391則文章以爬取完成
第2392則文章以爬取完成
第2393則文章以爬取完成
第2394則文章以爬取完成
第2395則文章以爬取完成
第2396則文章以爬取完成
第2397則文章以爬取完成
第2398則文章以爬取完成
第2399則文章以爬取完成
第2400則文章以爬取完成
第2401則文章以爬取完成
第2402則文章以爬取完成
第2403則文章以爬取完成
第2404則文章以爬取完成
第2405則文章以爬取完成
第2406則文章以爬取完成
第2407則文章以爬取完成
第2408則文章以爬取完成
第2409則文章以爬取完成
第2410則文章以爬取完成
第2411則文章以爬取完成
第2412則文章以爬取完成
第2413則文章以爬取完成
第2414則文章以爬取完成
第2415則文章以爬取完成
第2416則文章以爬取完成
第2417則文章以爬取完成
第2418則文章以爬取完成
第2419則文章以爬取完成
第2420則文章以爬取完成
第2421則文章以爬取完成
第2422則文章以爬取完成
第2423則文章以爬取完成
第2424則文章以爬取完成
第2425則文章以爬取完成
第2426則文章以爬取完成
第2427則文章以爬取完成
第2428則文章以爬取完成
第2429則文章以爬取完成
第2430則文章以爬取完成
第2431則文章以爬取完成
第2432則文章以爬取完成
第2433則文章以爬取完成
第2434則文章以爬取完成
第2435則文章以爬取完成
第2436則文章以爬取完成
第2437則文章以爬取完成
第2438則文章以爬取完成
第2439則文章以爬取完成
第2440則文章以爬取完成
第2441則文章以爬取完成
第2442則文章以爬取完成
第2443則文章以爬取完成
第2444則文章以爬取完成
第2445則文章以爬取完成
第2446則文章以爬取完成
第2447則文章以爬取完成
第2448則文章以爬取完成
第2449則文章以爬取完成
第2450則文章以爬取完成
第2451則

第2966則文章以爬取完成
第2967則文章以爬取完成
第2968則文章以爬取完成
第2969則文章以爬取完成
第2970則文章以爬取完成
第2971則文章以爬取完成
第2972則文章以爬取完成
第2973則文章以爬取完成
第2974則文章以爬取完成
第2975則文章以爬取完成
第2976則文章以爬取完成
第2977則文章以爬取完成
第2978則文章以爬取完成
第2979則文章以爬取完成
第2980則文章以爬取完成
第2981則文章以爬取完成
第2982則文章以爬取完成
第2983則文章以爬取完成
第2984則文章以爬取完成
第2985則文章以爬取完成
第2986則文章以爬取完成
第2987則文章以爬取完成
第2988則文章以爬取完成
第2989則文章以爬取完成
第2990則文章以爬取完成
第2991則文章以爬取完成
第2992則文章以爬取完成
第2993則文章以爬取完成
第2994則文章以爬取完成
第2995則文章以爬取完成
第2996則文章以爬取完成
第2997則文章以爬取完成
第2998則文章以爬取完成
第2999則文章以爬取完成
第3000則文章以爬取完成
第3001則文章以爬取完成
第3002則文章以爬取完成
第3003則文章以爬取完成
第3004則文章以爬取完成
第3005則文章以爬取完成
第3006則文章以爬取完成
第3007則文章以爬取完成
第3008則文章以爬取完成
第3009則文章以爬取完成
第3010則文章以爬取完成
第3011則文章以爬取完成
第3012則文章以爬取完成
第3013則文章以爬取完成
第3014則文章以爬取完成
第3015則文章以爬取完成
第3016則文章以爬取完成
第3017則文章以爬取完成
第3018則文章以爬取完成
第3019則文章以爬取完成
第3020則文章以爬取完成
第3021則文章以爬取完成
第3022則文章以爬取完成
第3023則文章以爬取完成
第3024則文章以爬取完成
第3025則文章以爬取完成
第3026則文章以爬取完成
第3027則文章以爬取完成
第3028則文章以爬取完成
第3029則文章以爬取完成
第3030則文章以爬取完成
第3031則文章以爬取完成
第3032則文章以爬取完成
第3033則文章以爬取完成
第3034則文章以爬取完成
第3035則文章以爬取完成
第3036則文章以爬取完成
第3037則

第3552則文章以爬取完成
第3553則文章以爬取完成
第3554則文章以爬取完成
第3555則文章以爬取完成
第3556則文章以爬取完成
第3557則文章以爬取完成
第3558則文章以爬取完成
第3559則文章以爬取完成
第3560則文章以爬取完成
第3561則文章以爬取完成
第3562則文章以爬取完成
第3563則文章以爬取完成
第3564則文章以爬取完成
第3565則文章以爬取完成
第3566則文章以爬取完成
第3567則文章以爬取完成
第3568則文章以爬取完成
第3569則文章以爬取完成
第3570則文章以爬取完成
第3571則文章以爬取完成
第3572則文章以爬取完成
第3573則文章以爬取完成
第3574則文章以爬取完成
第3575則文章以爬取完成
第3576則文章以爬取完成
第3577則文章以爬取完成
第3578則文章以爬取完成
第3579則文章以爬取完成
第3580則文章以爬取完成
第3581則文章以爬取完成
第3582則文章以爬取完成
第3583則文章以爬取完成
第3584則文章以爬取完成
第3585則文章以爬取完成
第3586則文章以爬取完成
第3587則文章以爬取完成
第3588則文章以爬取完成
第3589則文章以爬取完成
第3590則文章以爬取完成
第3591則文章以爬取完成
第3592則文章以爬取完成
第3593則文章以爬取完成
第3594則文章以爬取完成
第3595則文章以爬取完成
第3596則文章以爬取完成
第3597則文章以爬取完成
第3598則文章以爬取完成
第3599則文章以爬取完成
第3600則文章以爬取完成
第3601則文章以爬取完成
第3602則文章以爬取完成
第3603則文章以爬取完成
第3604則文章以爬取完成
第3605則文章以爬取完成
第3606則文章以爬取完成
第3607則文章以爬取完成
第3608則文章以爬取完成
第3609則文章以爬取完成
第3610則文章以爬取完成
第3611則文章以爬取完成
第3612則文章以爬取完成
第3613則文章以爬取完成
第3614則文章以爬取完成
第3615則文章以爬取完成
第3616則文章以爬取完成
第3617則文章以爬取完成
第3618則文章以爬取完成
第3619則文章以爬取完成
第3620則文章以爬取完成
第3621則文章以爬取完成
第3622則文章以爬取完成
第3623則

...

第5310則文章以爬取完成
第5311則文章以爬取完成
第5312則文章以爬取完成
第5313則文章以爬取完成
第5314則文章以爬取完成
第5315則文章以爬取完成
第5316則文章以爬取完成
第5317則文章以爬取完成
第5318則文章以爬取完成
第5319則文章以爬取完成
第5320則文章以爬取完成
第5321則文章以爬取完成
第5322則文章以爬取完成
第5323則文章以爬取完成
第5324則文章以爬取完成
第5325則文章以爬取完成
第5326則文章以爬取完成
第5327則文章以爬取完成
第5328則文章以爬取完成
第5329則文章以爬取完成
第5330則文章以爬取完成
第5331則文章以爬取完成
第5332則文章以爬取完成
第5333則文章以爬取完成
第5334則文章以爬取完成
第5335則文章以爬取完成
第5336則文章以爬取完成
第5337則文章以爬取完成
第5338則文章以爬取完成
第5339則文章以爬取完成
第5340則文章以爬取完成
第5341則文章以爬取完成
第5342則文章以爬取完成
第5343則文章以爬取完成
第5344則文章以爬取完成
第5345則文章以爬取完成
第5346則文章以爬取完成
第5347則文章以爬取完成
第5348則文章以爬取完成
第5349則文章以爬取完成
第5350則文章以爬取完成
第5351則文章以爬取完成
第5352則文章以爬取完成
第5353則文章以爬取完成
第5354則文章以爬取完成
第5355則文章以爬取完成
第5356則文章以爬取完成
第5357則文章以爬取完成
第5358則文章以爬取完成
第5359則文章以爬取完成
第5360則文章以爬取完成
第5361則文章以爬取完成
第5362則文章以爬取完成
第5363則文章以爬取完成
第5364則文章以爬取完成
第5365則文章以爬取完成
第5366則文章以爬取完成
第5367則文章以爬取完成
第5368則文章以爬取完成
第5369則文章以爬取完成
第5370則文章以爬取完成
第5371則文章以爬取完成
第5372則文章以爬取完成
第5373則文章以爬取完成
第5374則文章以爬取完成
第5375則文章以爬取完成
第5376則文章以爬取完成
第5377則文章以爬取完成
第5378則文章以爬取完成
第5379則文章以爬取完成
第5380則文章以爬取完成
第5381則

第5896則文章以爬取完成
第5897則文章以爬取完成
第5898則文章以爬取完成
第5899則文章以爬取完成
第5900則文章以爬取完成
第5901則文章以爬取完成
第5902則文章以爬取完成
第5903則文章以爬取完成
第5904則文章以爬取完成
第5905則文章以爬取完成
第5906則文章以爬取完成
第5907則文章以爬取完成
第5908則文章以爬取完成
第5909則文章以爬取完成
第5910則文章以爬取完成
第5911則文章以爬取完成
第5912則文章以爬取完成
第5913則文章以爬取完成
第5914則文章以爬取完成
第5915則文章以爬取完成
第5916則文章以爬取完成
第5917則文章以爬取完成
第5918則文章以爬取完成
第5919則文章以爬取完成
第5920則文章以爬取完成
第5921則文章以爬取完成
第5922則文章以爬取完成
第5923則文章以爬取完成
第5924則文章以爬取完成
第5925則文章以爬取完成
第5926則文章以爬取完成
第5927則文章以爬取完成
第5928則文章以爬取完成
第5929則文章以爬取完成
第5930則文章以爬取完成
第5931則文章以爬取完成
第5932則文章以爬取完成
第5933則文章以爬取完成
第5934則文章以爬取完成
第5935則文章以爬取完成
第5936則文章以爬取完成
第5937則文章以爬取完成
第5938則文章以爬取完成
第5939則文章以爬取完成
第5940則文章以爬取完成
第5941則文章以爬取完成
第5942則文章以爬取完成
第5943則文章以爬取完成
第5944則文章以爬取完成
第5945則文章以爬取完成
第5946則文章以爬取完成
第5947則文章以爬取完成
第5948則文章以爬取完成
第5949則文章以爬取完成
第5950則文章以爬取完成
第5951則文章以爬取完成
第5952則文章以爬取完成
第5953則文章以爬取完成
第5954則文章以爬取完成
第5955則文章以爬取完成
第5956則文章以爬取完成
第5957則文章以爬取完成
第5958則文章以爬取完成
第5959則文章以爬取完成
第5960則文章以爬取完成
第5961則文章以爬取完成
第5962則文章以爬取完成
第5963則文章以爬取完成
第5964則文章以爬取完成
第5965則文章以爬取完成
第5966則文章以爬取完成
第5967則

第6482則文章以爬取完成
第6483則文章以爬取完成
第6484則文章以爬取完成
第6485則文章以爬取完成
第6486則文章以爬取完成
第6487則文章以爬取完成
第6488則文章以爬取完成
第6489則文章以爬取完成
第6490則文章以爬取完成
第6491則文章以爬取完成
第6492則文章以爬取完成
第6493則文章以爬取完成
第6494則文章以爬取完成
第6495則文章以爬取完成
第6496則文章以爬取完成
第6497則文章以爬取完成
第6498則文章以爬取完成
第6499則文章以爬取完成
第6500則文章以爬取完成
第6501則文章以爬取完成
第6502則文章以爬取完成
第6503則文章以爬取完成
第6504則文章以爬取完成
第6505則文章以爬取完成
第6506則文章以爬取完成
第6507則文章以爬取完成
第6508則文章以爬取完成
第6509則文章以爬取完成
第6510則文章以爬取完成
第6511則文章以爬取完成
第6512則文章以爬取完成
第6513則文章以爬取完成
第6514則文章以爬取完成
第6515則文章以爬取完成
第6516則文章以爬取完成
第6517則文章以爬取完成
第6518則文章以爬取完成
第6519則文章以爬取完成
第6520則文章以爬取完成
第6521則文章以爬取完成
第6522則文章以爬取完成
第6523則文章以爬取完成
第6524則文章以爬取完成
第6525則文章以爬取完成
第6526則文章以爬取完成
第6527則文章以爬取完成
第6528則文章以爬取完成
第6529則文章以爬取完成
第6530則文章以爬取完成
第6531則文章以爬取完成
第6532則文章以爬取完成
第6533則文章以爬取完成
第6534則文章以爬取完成
第6535則文章以爬取完成
第6536則文章以爬取完成
第6537則文章以爬取完成
第6538則文章以爬取完成
第6539則文章以爬取完成
第6540則文章以爬取完成
第6541則文章以爬取完成
第6542則文章以爬取完成
第6543則文章以爬取完成
第6544則文章以爬取完成
第6545則文章以爬取完成
第6546則文章以爬取完成
第6547則文章以爬取完成
第6548則文章以爬取完成
第6549則文章以爬取完成
第6550則文章以爬取完成
第6551則文章以爬取完成
第6552則文章以爬取完成
第6553則

第7068則文章以爬取完成
第7069則文章以爬取完成
第7070則文章以爬取完成
第7071則文章以爬取完成
第7072則文章以爬取完成
第7073則文章以爬取完成
第7074則文章以爬取完成
第7075則文章以爬取完成
第7076則文章以爬取完成
第7077則文章以爬取完成
第7078則文章以爬取完成
第7079則文章以爬取完成
第7080則文章以爬取完成
第7081則文章以爬取完成
第7082則文章以爬取完成
第7083則文章以爬取完成
第7084則文章以爬取完成
第7085則文章以爬取完成
第7086則文章以爬取完成
第7087則文章以爬取完成
第7088則文章以爬取完成
第7089則文章以爬取完成
第7090則文章以爬取完成
第7091則文章以爬取完成
第7092則文章以爬取完成
第7093則文章以爬取完成
第7094則文章以爬取完成
第7095則文章以爬取完成
第7096則文章以爬取完成
第7097則文章以爬取完成
第7098則文章以爬取完成
第7099則文章以爬取完成
第7100則文章以爬取完成
第7101則文章以爬取完成
第7102則文章以爬取完成
第7103則文章以爬取完成
第7104則文章以爬取完成
第7105則文章以爬取完成
第7106則文章以爬取完成
第7107則文章以爬取完成
第7108則文章以爬取完成
第7109則文章以爬取完成
第7110則文章以爬取完成
第7111則文章以爬取完成
第7112則文章以爬取完成
第7113則文章以爬取完成
第7114則文章以爬取完成
第7115則文章以爬取完成
第7116則文章以爬取完成
第7117則文章以爬取完成
第7118則文章以爬取完成
第7119則文章以爬取完成
第7120則文章以爬取完成
第7121則文章以爬取完成
第7122則文章以爬取完成
第7123則文章以爬取完成
第7124則文章以爬取完成
第7125則文章以爬取完成
第7126則文章以爬取完成
第7127則文章以爬取完成
第7128則文章以爬取完成
第7129則文章以爬取完成
第7130則文章以爬取完成
第7131則文章以爬取完成
第7132則文章以爬取完成
第7133則文章以爬取完成
第7134則文章以爬取完成
第7135則文章以爬取完成
第7136則文章以爬取完成
第7137則文章以爬取完成
第7138則文章以爬取完成
第7139則

第7654則文章以爬取完成
第7655則文章以爬取完成
第7656則文章以爬取完成
第7657則文章以爬取完成
第7658則文章以爬取完成
第7659則文章以爬取完成
第7660則文章以爬取完成
第7661則文章以爬取完成
第7662則文章以爬取完成
第7663則文章以爬取完成
第7664則文章以爬取完成
第7665則文章以爬取完成
第7666則文章以爬取完成
第7667則文章以爬取完成
第7668則文章以爬取完成
第7669則文章以爬取完成
第7670則文章以爬取完成
第7671則文章以爬取完成
第7672則文章以爬取完成
第7673則文章以爬取完成
第7674則文章以爬取完成
第7675則文章以爬取完成
第7676則文章以爬取完成
第7677則文章以爬取完成
第7678則文章以爬取完成
第7679則文章以爬取完成
第7680則文章以爬取完成
第7681則文章以爬取完成
第7682則文章以爬取完成
第7683則文章以爬取完成
第7684則文章以爬取完成
第7685則文章以爬取完成
第7686則文章以爬取完成
第7687則文章以爬取完成
第7688則文章以爬取完成
第7689則文章以爬取完成
第7690則文章以爬取完成
第7691則文章以爬取完成
第7692則文章以爬取完成
第7693則文章以爬取完成
第7694則文章以爬取完成
第7695則文章以爬取完成
第7696則文章以爬取完成
第7697則文章以爬取完成
第7698則文章以爬取完成
第7699則文章以爬取完成
第7700則文章以爬取完成
第7701則文章以爬取完成
第7702則文章以爬取完成


In [94]:
### use os method to get the file from workspace directory ##
import  os
DATA_DIR = "C:\\Users\\Java\\Desktop\\mobile01_JP\\"
file_data = []
for filename in os.listdir(DATA_DIR):
    print( "Loading: {}".format(filename))
    loadFile = open(os.path.join(DATA_DIR, filename), 'r', encoding='UTF-8')
    file_data.append(loadFile.read())
    loadFile.close()

Loading: command1.text
Loading: command10.text
Loading: command100.text
Loading: command1000.text
Loading: command1001.text
Loading: command1002.text
Loading: command1003.text
Loading: command1004.text
Loading: command1005.text
Loading: command1006.text
Loading: command1007.text
Loading: command1008.text
Loading: command1009.text
Loading: command101.text
Loading: command1010.text
Loading: command1011.text
Loading: command1012.text
Loading: command1013.text
Loading: command1014.text
Loading: command1015.text
Loading: command1016.text
Loading: command1017.text
Loading: command1018.text
Loading: command1019.text
Loading: command102.text
Loading: command1020.text
Loading: command1021.text
Loading: command1022.text
Loading: command1023.text
Loading: command1024.text
Loading: command1025.text
Loading: command1026.text
Loading: command1027.text
Loading: command1028.text
Loading: command1029.text
Loading: command103.text
Loading: command1030.text
Loading: command1031.text
Loading: command1032.

Loading: command195.text
Loading: command1950.text
Loading: command1951.text
Loading: command1952.text
Loading: command1953.text
Loading: command1954.text
Loading: command1955.text
Loading: command1956.text
Loading: command1957.text
Loading: command1958.text
Loading: command1959.text
Loading: command196.text
Loading: command1960.text
Loading: command1961.text
Loading: command1962.text
Loading: command1963.text
Loading: command1964.text
Loading: command1965.text
Loading: command1966.text
Loading: command1967.text
Loading: command1968.text
Loading: command1969.text
Loading: command197.text
Loading: command1970.text
Loading: command1971.text
Loading: command1972.text
Loading: command1973.text
Loading: command1974.text
Loading: command1975.text
Loading: command1976.text
Loading: command1977.text
Loading: command1978.text
Loading: command1979.text
Loading: command198.text
Loading: command1980.text
Loading: command1981.text
Loading: command1982.text
Loading: command1983.text
Loading: command

Loading: command2995.text
Loading: command2996.text
Loading: command2997.text
Loading: command2998.text
Loading: command2999.text
Loading: command3.text
Loading: command30.text
Loading: command300.text
Loading: command3000.text
Loading: command3001.text
Loading: command3002.text
Loading: command3003.text
Loading: command3004.text
Loading: command3005.text
Loading: command3006.text
Loading: command3007.text
Loading: command3008.text
Loading: command3009.text
Loading: command301.text
Loading: command3010.text
Loading: command3011.text
Loading: command3012.text
Loading: command3013.text
Loading: command3014.text
Loading: command3015.text
Loading: command3016.text
Loading: command3017.text
Loading: command3018.text
Loading: command3019.text
Loading: command302.text
Loading: command3020.text
Loading: command3021.text
Loading: command3022.text
Loading: command3023.text
Loading: command3024.text
Loading: command3025.text
Loading: command3026.text
Loading: command3027.text
Loading: command3028

Loading: command4069.text
Loading: command407.text
Loading: command4070.text
Loading: command4071.text
Loading: command4072.text
Loading: command4073.text
Loading: command4074.text
Loading: command4075.text
Loading: command4076.text
Loading: command4077.text
Loading: command4078.text
Loading: command4079.text
Loading: command408.text
Loading: command4080.text
Loading: command4081.text
Loading: command4082.text
Loading: command4083.text
Loading: command4084.text
Loading: command4085.text
Loading: command4086.text
Loading: command4087.text
Loading: command4088.text
Loading: command4089.text
Loading: command409.text
Loading: command4090.text
Loading: command4091.text
Loading: command4092.text
Loading: command4093.text
Loading: command4094.text
Loading: command4095.text
Loading: command4096.text
Loading: command4097.text
Loading: command4098.text
Loading: command4099.text
Loading: command41.text
Loading: command410.text
Loading: command4100.text
Loading: command4101.text
Loading: command41

Loading: command522.text
Loading: command5220.text
Loading: command5221.text
Loading: command5222.text
Loading: command5223.text
Loading: command5224.text
Loading: command5225.text
Loading: command5226.text
Loading: command5227.text
Loading: command5228.text
Loading: command5229.text
Loading: command523.text
Loading: command5230.text
Loading: command5231.text
Loading: command5232.text
Loading: command5233.text
Loading: command5234.text
Loading: command5235.text
Loading: command5236.text
Loading: command5237.text
Loading: command5238.text
Loading: command5239.text
Loading: command524.text
Loading: command5240.text
Loading: command5241.text
Loading: command5242.text
Loading: command5243.text
Loading: command5244.text
Loading: command5245.text
Loading: command5246.text
Loading: command5247.text
Loading: command5248.text
Loading: command5249.text
Loading: command525.text
Loading: command5250.text
Loading: command5251.text
Loading: command5252.text
Loading: command5253.text
Loading: command

Loading: command6387.text
Loading: command6388.text
Loading: command6389.text
Loading: command639.text
Loading: command6390.text
Loading: command6391.text
Loading: command6392.text
Loading: command6393.text
Loading: command6394.text
Loading: command6395.text
Loading: command6396.text
Loading: command6397.text
Loading: command6398.text
Loading: command6399.text
Loading: command64.text
Loading: command640.text
Loading: command6400.text
Loading: command6401.text
Loading: command6402.text
Loading: command6403.text
Loading: command6404.text
Loading: command6405.text
Loading: command6406.text
Loading: command6407.text
Loading: command6408.text
Loading: command6409.text
Loading: command641.text
Loading: command6410.text
Loading: command6411.text
Loading: command6412.text
Loading: command6413.text
Loading: command6414.text
Loading: command6415.text
Loading: command6416.text
Loading: command6417.text
Loading: command6418.text
Loading: command6419.text
Loading: command642.text
Loading: command64

Loading: command7446.text
Loading: command7447.text
Loading: command7448.text
Loading: command7449.text
Loading: command745.text
Loading: command7450.text
Loading: command7451.text
Loading: command7452.text
Loading: command7453.text
Loading: command7454.text
Loading: command7455.text
Loading: command7456.text
Loading: command7457.text
Loading: command7458.text
Loading: command7459.text
Loading: command746.text
Loading: command7460.text
Loading: command7461.text
Loading: command7462.text
Loading: command7463.text
Loading: command7464.text
Loading: command7465.text
Loading: command7466.text
Loading: command7467.text
Loading: command7468.text
Loading: command7469.text
Loading: command747.text
Loading: command7470.text
Loading: command7471.text
Loading: command7472.text
Loading: command7473.text
Loading: command7474.text
Loading: command7475.text
Loading: command7476.text
Loading: command7477.text
Loading: command7478.text
Loading: command7479.text
Loading: command748.text
Loading: command

In [106]:
from pymongo import MongoClient
client = MongoClient('10.120.27.11', 27017)

In [132]:
for i in file_data:
    i = json.loads(i)
    
    client.text_mining.mobile01_article.insert_one(i)

In [111]:
len(file_data)

7702